In [33]:
# %pip install tiktoken openai --quiet --upgrade

# Managing Conversation State with OpenAI's API

In this notebook, we'll walk through how to manually manage conversation state when interacting with the OpenAI API. We'll cover two approaches:

1. **Manually managing conversation state:** Building a conversation history list and appending messages for each turn.
2. **Chaining responses using `previous_response_id`:** This method allows the model to access prior context automatically.

Let's get started!

## 1. Manual Conversation State Management

For this first example, we'll create a simple knock-knock joke conversation by manually constructing a conversation history. Every turn in the conversation is stored in a list of messages, where each message is a dictionary containing a role (`user` or `assistant`) and the content.

This way of managing state is very explicit and works well when you want fine-grained control over the conversation history.

In [34]:
from openai import OpenAI

In [35]:
MODEL = "gpt-4.1-mini"

In [ ]:
import os
# Create an OpenAI client instance
client = OpenAI(
    # Replace with your actual API key or use: api_key=os.environ.get("OPENAI_API_KEY")
    api_key=os.environ.get("OPENAI_API_KEY_PRACTICE")
)

print(os.environ.get("OPENAI_API_KEY_PRACTICE"))

C:\Program Files\Java\jdk-21


In [37]:
# Construct a conversation history manually for a knock-knock joke
history = [
    {"role": "user", "content": "knock knock."},
    {"role": "assistant", "content": "Who's there?"},
    {"role": "user", "content": "Orange."}
]

# Create a response using the conversation history
#maintain continuity of the conversation. by sending the entire conversation context (previous user and model messages) 
response = client.responses.create(
    model=MODEL,
    input=history
)

print("Assistant response:", response.output_text)

# (Optional) Inspect the full response object for further details
# print(response)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: ctx7sk-9*******************************8aa3. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

## 2. Updating Conversation History Across Rounds

In this example, we'll start a conversation with a simple joke prompt. We'll then update the conversation history with the model's response, and ask for another joke. Updating the conversation history manually makes sure the model has full context of previous turns.

Let's see how this works.

In [ ]:
# Start a new conversation with an initial joke request
history = [
    {"role": "user", "content": "tell me a science joke"}
]

response = client.responses.create(
    model=MODEL,
    input=history,
    store=False  # We're not storing the conversation automatically
)

print("Joke:", response.output_text)

# Update the conversation history with all parts of the response
history += [{"role": el.role, "content": el.content} for el in response.output]

# Ask for another joke in the same conversation
history.append({"role": "user", "content": "tell me another"})

second_response = client.responses.create(
    model=MODEL,
    input=history,
    store=False
)

print("Another joke:", second_response.output_text)

Joke: Sure! Here's a sports joke for you:

Why did the soccer player bring a ladder to the game?  
Because they wanted to go to the next level! 😄⚽
Another joke: Of course! Here's another one:

Why did the basketball team go to the bank?  
Because they wanted to get their rebound! 🏀😄


## 3. Chaining Responses with `previous_response_id`

Another way to carry conversation context is to chain responses by using the `previous_response_id` parameter. With this method, you initiate a conversation and then provide additional requests that reference the previous response.

Below is an example where we first ask the model for a joke and then ask it to explain why that joke is funny.

In [ ]:
# Ask for a joke
response = client.responses.create(
    model=MODEL,
    input="tell me a joke"
)

print("Joke:", response.output_text)

# Chain the next request using previous_response_id so the model retains context
second_response = client.responses.create(
    model=MODEL,
    previous_response_id=response.id,
    input=[{"role": "user", "content": "explain why this is funny."}]
)

print("Explanation:", second_response.output_text)

Joke: Sure! Here’s a joke for you:

Why did the scarecrow win an award?

Because he was outstanding in his field! 😄
Explanation: Absolutely! This joke is funny because it uses a play on words, or a pun.

- The phrase "outstanding in his field" literally means the scarecrow is physically standing out in a field, which is what scarecrows do—they stand in fields to scare away birds.
- At the same time, "outstanding in his field" is a common expression meaning someone is very good or excellent at their profession or area of expertise.

The humor comes from combining these two meanings—the literal scene of a scarecrow standing in a field and the figurative praise for being excellent—creating a clever twist that makes the joke amusing.


## Final Notes

- **Managing Tokens:** Keep in mind that every input and output token (and any internal reasoning tokens) counts toward your model's context window. For long conversations, you may need to manage or truncate history.
- **Best Practices:** When building production applications, consider automating state management so that conversations flow naturally. Using proper error handling, logging, and context management will improve your application's DX significantly.

This notebook demonstrated how you can use Python to manage conversation state. Feel free to experiment with other conversation flows and adjust the approach as needed for your teaching or production environments.